In [1]:
import pandas as pd
import numpy as np
import ipywidgets as w
from IPython.display import HTML
import bqplot as bq
import copy
import traitlets as tr

In [2]:
MAP_COLORS = {'Heating': '#c22324',
 'Domestic Hot Water': '#ff7f0e',
 'Heating & Domestic Hot Water': '#eb4f18',
 'Cooling': '#1f77b4',
 'Auxiliary': '#9467bd',
 'Lighting': '#17becf',
 'Small Power': '#d3d3d3',
 'IT/Servers': '#8c564b',
 'PV Generation': '#bcbd22',
 'Catering': '#2ca02c',
 'Lifts': '#e377c2',
 'Other': '#000000',
 'Unknown': '#7C7270'}

HEADER = """<h1>NHS Archetypes - Decarbonisation Pathways Analysis and Visualisation</h1>

<ul>
  <li>This app is a dummy demo, illustrating the type of front-end that can be expected for visualising different decarbonisation pathways</li>
  <li>It runs on <code>python</code> code in the browser and can be deployed as a static HTML website</li>
  <li>This is an illustrative example but the calculation would be developed to consider the different archetypes within the NHS portfolio</li>
  <li>Choose from the possible design interventions to understand the incremental savings that are available (the numbers are made up)</li>
</ul> 
"""

class EnergyConsumption(w.VBox):     
    total_saving = tr.Float()

    @tr.observe("total_saving")
    def obs_total_saving(self, on_change):
        self.html_total_saving.value = f"<b>Total Saving</b> = {self.total_saving}%"
        
    def __init__(self, data: dict[str, np.array]):
        if "existing building" not in data:
            raise ValueError("`existing building` data missing.")
        self.data = data
        options = [k for k in data.keys() if k != "existing building"]
        self.html_total_saving = w.HTML()
        self.select = w.SelectMultiple(options=options, layout={"height": "150px", "width": "400px"})
        self.vbx_select = w.VBox([w.HTML("<b>Select multiple possible the design interventions:</b>"), self.select])
        self.hbx_topbar = w.HBox([w.VBox([w.HTML(HEADER), self.html_total_saving]), self.vbx_select], layout={"justify_content":"space-between"})
        self.xs = bq.OrdinalScale()  # ordinal scale to represent categorical data
        self.ys = bq.LinearScale()
        self.xax = bq.Axis(scale=self.xs, grid_lines="none")  # no grid lines needed for x
        self.yax = bq.Axis(
            scale=self.ys, orientation="vertical", label="Energy Consumption (kWh/m2/yr)"
        )
        self.fig = bq.Figure(title="Annual Energy Consumption", axes=[self.xax, self.yax], legend_location="top-right")#
        super().__init__([self.hbx_topbar, self.fig])
        self._init_controls()
        self._set_chart()

    def _init_controls(self):
        self.select.observe(self._set_chart)
    
    def _get_y(self, data: dict[str, np.array]) -> np.array:
        data_copy = copy.deepcopy(data)
        baseline = data_copy.pop("existing building")
        arrays = np.array(list(data_copy.values()))
        input_y = np.array([baseline])
        for idx, array in enumerate(arrays):
            diff = input_y[-1] - arrays[idx]
            input_y = np.vstack([input_y, diff])
        return input_y

    @property
    def filtered_data(self):
        return {"existing building": self.data["existing building"]} | {k: v for k, v in self.data.items() if k in self.select.value}
    
    @property
    def calc_y(self):
        return self._get_y(self.filtered_data)
        
    def _set_chart(self, onchange=None):
        self.bar = bq.Bars(
            x=np.array(list(self.filtered_data.keys())),
            y=np.transpose(self.calc_y),
            scales={"x": self.xs, "y": self.ys},
            type="stacked",
            colors=colors,
            labels=labels,
            display_legend=True,
            padding=0.4,
        )
        self.xs.domain = ["existing building"] + list(self.select.value)
        self.fig.marks = [self.bar]
        self._update_saving()

    def _update_saving(self):
        y = self.calc_y
        self.total_saving = np.round((1 - (y[-1].sum() / y[0].sum())) * 100, 0)
        
labels = ['Heating', 'Domestic Hot Water', 'Auxiliary', 'Lighting', 'Small Power', 'PV Generation']
colors = [MAP_COLORS[x] for x in labels]
data = {
    "existing building": np.array([25, 23, 11, 13, 15, 0]),
    "replace glazing": np.array([10, 0, 0, 0, 0, 0]),
    "improve airtightness": np.array([5, 0, 0, 0, 0, 0]),
    "add efficient heat source": np.array([5, 12, 0, 0, 0, 0]),
    "improve controls": np.array([0, 0, 3, 0, 0, 0]),
    "update lighting to LED": np.array([0, 0, 0, 5, 0, 0]),
    "add PV panels": np.array([0, 0, 0, 0,  0, 10]),
}

chart = EnergyConsumption(data)
display(chart)

EnergyConsumption(children=(HBox(children=(VBox(children=(HTML(value='<h1>NHS Archetypes - Decarbonisation Pat…